In [ ]:
import numpy as np
import cv2 as cv
import glob
import os

from google.colab.patches import cv2_imshow

# Función para cargar y aumentar imágenes
def augment_images(image_paths, scale_factor=1.2, rotation_angle=10):
    augmented_images = []
    for path in image_paths:
        image = cv.imread(path)
        augmented_images.append(image)

        # Escala
        scaled_image = cv.resize(image, None, fx=scale_factor, fy=scale_factor, interpolation=cv.INTER_LINEAR)
        augmented_images.append(scaled_image)

        # Rotación
        rows, cols, _ = image.shape
        rotation_matrix = cv.getRotationMatrix2D((cols/2, rows/2), rotation_angle, 1)
        rotated_image = cv.warpAffine(image, rotation_matrix, (cols, rows))
        augmented_images.append(rotated_image)

    return augmented_images

# Función para mostrar imágenes
def show_images(images):
    for idx, img in enumerate(images):
        cv2_imshow(img)
        cv.waitKey(500)

# criterios de terminación
criterios = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# preparar puntos de objeto, como (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Matrices para almacenar puntos de objetos y puntos de imágenes de todas las imágenes.
objpoints = [] # punto 3d en el espacio del mundo real
imgpoints = [] # puntos 2d en el plano de la imagen.

image_paths = glob.glob("/content/drive/MyDrive/UNIVERSITIES/EAFIT University (1)/2024-1/Visión por Computador en la Agricultura - IA0105/s10/left/*.jpeg")


ruta_carpeta = '/content/drive/MyDrive/UNIVERSITIES/EAFIT University (1)/2024-1/Visión por Computador en la Agricultura - IA0105/s10/left'
imagenes = glob.glob(os.path.join(ruta_carpeta, '*.jpeg'))

# Verificar si hay al menos una imagen en la lista
if len(imagenes) > 0:
    # Aumentar las imágenes
    all_images = augment_images(imagenes)

    # Mostrar imágenes originales y aumentadas
    show_images(all_images)

    for img_path in imagenes:
        img = cv.imread(img_path)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        # Encuentra las esquinas del tablero de ajedrez.
        ret, corners = cv.findChessboardCorners(gray, (7,6), None)

        # Si lo encuentra, agregue puntos de objeto, puntos de imagen (después de refinarlos)
        if ret == True:
            objpoints.append(objp)

            corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criterios)
            imgpoints.append(corners2)

            # Dibuja y muestra las esquinas.
            img_drawn = img.copy()
            cv.drawChessboardCorners(img_drawn, (7,6), corners2, ret)
            cv2_imshow(img_drawn)
            cv.waitKey(500)

    # Calibración de la cámara
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    if ret:
        print("La calibración de la cámara fue exitosa.")
    else:
        print("La calibración de la cámara falló.")
else:
    print("No se encontraron imágenes en la carpeta especificada.")


In [ ]:
# Emparejar puntos de características entre dos imágenes
def match_features(image1, image2):
    # Convertir imágenes a escala de grises
    gray1 = cv.cvtColor(image1, cv.COLOR_BGR2GRAY)
    gray2 = cv.cvtColor(image2, cv.COLOR_BGR2GRAY)

    # Inicializar el detector de características y el descriptor
    orb = cv.ORB_create()

    # Encontrar puntos clave y descriptores en las imágenes
    kp1, des1 = orb.detectAndCompute(gray1, None)
    kp2, des2 = orb.detectAndCompute(gray2, None)

    # Inicializar el matcher de fuerza bruta
    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

    # Hacer coincidir descriptores entre las dos imágenes
    matches = bf.match(des1, des2)

    # Extraer coordenadas de puntos coincidentes
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    return pts1, pts2

# Calcular la matriz esencial entre dos imágenes
def calculate_essential_matrix(pts1, pts2, mtx):
    # Normalizar los puntos de imagen
    pts1_norm = cv.undistortPoints(pts1, mtx, None)
    pts2_norm = cv.undistortPoints(pts2, mtx, None)

    # Calcular la matriz esencial
    F, mask = cv.findFundamentalMat(pts1_norm, pts2_norm, cv.FM_LMEDS)
    E = mtx.T @ F @ mtx

    return E

# Calcular la matriz de proyección de la segunda cámara
def calculate_projection_matrix(E, pts1, pts2):
    _, R, t, _ = cv.recoverPose(E, pts1, pts2)
    P2 = np.hstack((R, t))

    return P2

# Triangulación para calcular las coordenadas 3D
def triangulate_points(pts1, pts2, P1, P2):
    # Triangulación de puntos
    points_4d = cv.triangulatePoints(P1, P2, pts1, pts2)

    # Convertir coordenadas homogéneas a coordenadas cartesianas
    points_3d = cv.convertPointsFromHomogeneous(points_4d.T)

    return points_3d.reshape(-1, 3)

In [ ]:
# Calcular la matriz de proyección de la segunda cámara
def calculate_projection_matrix(E, pts1, pts2):
    _, R, t, _ = cv.recoverPose(E, pts1, pts2)
    R1 = np.eye(3)
    t1 = np.zeros((3, 1))
    P1 = np.hstack((R1, t1))
    P2 = np.hstack((R, t.reshape(-1, 1)))
    return P1, P2

# Reconstrucción 3D a partir de varias imágenes
def reconstruct_3d_multiple_images(imgpaths, mtx, dist):
    # Inicializar la lista de puntos 3D reconstruidos
    reconstructed_points = []

    # Iterar sobre pares de imágenes adyacentes
    for i in range(len(imgpaths) - 1):
        # Cargar imágenes
        img1 = cv.imread(imgpaths[i])
        img2 = cv.imread(imgpaths[i+1])

        # Encontrar puntos de características coincidentes
        pts1, pts2 = match_features(img1, img2)

        # Calcular la matriz esencial
        E = calculate_essential_matrix(pts1, pts2, mtx)

        # Calcular las matrices de proyección de ambas cámaras
        P1, P2 = calculate_projection_matrix(E, pts1, pts2)

        # Triangulación de puntos
        points_3d = triangulate_points(pts1, pts2, P1, P2)

        # Agregar puntos 3D reconstruidos a la lista
        reconstructed_points.append(points_3d)

    return reconstructed_points

# Reconstrucción 3D
reconstructed_points = reconstruct_3d_multiple_images(imagenes, mtx, dist)


In [ ]:
reconstructed_points

[array([[ 9.56452429e-01,  5.46087436e-02,  1.58997811e-03],
        [ 9.30612981e-01, -1.01698928e-01,  1.78363954e-03],
        [ 9.85465586e-01,  2.22844884e-01,  2.33771978e-03],
        [ 9.84407961e-01,  2.05567226e-01,  3.18427663e-03],
        [ 9.55837131e-01,  4.71038222e-02,  1.96306687e-03],
        [ 9.59791124e-01,  6.59915060e-02,  2.10232986e-03],
        [ 9.17586207e-01, -1.85433239e-01,  2.14620959e-03],
        [ 9.87363160e-01,  2.35088125e-01,  2.20831437e-03],
        [ 9.95552957e-01,  2.67764658e-01,  2.01876275e-03],
        [ 9.65621650e-01,  9.75924432e-02,  2.38629919e-03],
        [ 9.55025315e-01,  4.05395180e-02,  1.85566861e-03],
        [ 9.69299078e-01,  1.23249158e-01,  2.56368797e-03],
        [ 9.46882069e-01, -5.50722657e-03,  1.60837499e-03],
        [ 3.75073457e+00,  1.64450455e+01,  3.57223675e-02],
        [ 9.30017233e-01, -1.05731696e-01,  2.48834537e-03],
        [ 9.76508260e-01,  1.67962238e-01,  2.41128565e-03],
        [ 9.25042629e-01